In [13]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()

# Here, we just set how many documents we'll process for automatic testing- you can safely ignore this!
n_docs = 500 if 'CI' in os.environ else 2591

from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

train_cands = session.query(Spouse).filter(Spouse.split == 0).order_by(Spouse.id).all()
dev_cands   = session.query(Spouse).filter(Spouse.split == 1).order_by(Spouse.id).all()
test_cands  = session.query(Spouse).filter(Spouse.split == 2).order_by(Spouse.id).all()
print(len(train_cands),len(dev_cands))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
22276 2814


In [23]:
from util import load_external_labels

# %time load_external_labels(session, Spouse, annotator_name='gold')

from snorkel.annotations import load_gold_labels

#L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
#L_gold_test = load_gold_labels(session, annotator_name='gold', split=2, zero_one=True)

# L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
# L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)
L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
# gold_labels_dev = [L[0,0] if L[0,0]==1 else -1 for L in L_gold_dev]
gold_labels_dev = [L[0,0] for L in L_gold_dev]



from snorkel.learning.utils import MentionScorer

In [5]:
#gold_labels_dev = [x[0,0] for x in L_gold_dev.todense()]
#for i,L in enumerate(gold_labels_dev):
#    print(i,gold_labels_dev[i])

# gold_labels_dev = []
# for i,L in enumerate(L_gold_dev):
#     gold_labels_dev.append(L[0,0])
    
# gold_labels_test = []
# for i,L in enumerate(L_gold_test):
#     gold_labels_test.append(L[0,0])
    
# print(len(gold_labels_dev),len(gold_labels_test))
print(gold_labels_dev.count(1),gold_labels_dev.count(-1))
print(len(gold_labels_dev))

189 2625
2814


In [6]:
import re
from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

In [7]:
####### Discrete ##########

spouses = {'spouse', 'wife', 'husband', 'ex-wife', 'ex-husband'}
family = {'father', 'mother', 'sister', 'brother', 'son', 'daughter',
              'grandfather', 'grandmother', 'uncle', 'aunt', 'cousin'}
family = family | {f + '-in-law' for f in family}
other = {'boyfriend', 'girlfriend' 'boss', 'employee', 'secretary', 'co-worker'}

# Helper function to get last name
def last_name(s):
    name_parts = s.split(' ')
    return name_parts[-1] if len(name_parts) > 1 else None    

def LF_husband_wife(c):
    return (1,1) if len(spouses.intersection(get_between_tokens(c))) > 0 else (0,0)

def LF_husband_wife_left_window(c):
    if len(spouses.intersection(get_left_tokens(c[0], window=2))) > 0:
        return (1,1)
    elif len(spouses.intersection(get_left_tokens(c[1], window=2))) > 0:
        return (1,1)
    else:
        return (0,0)
    
def LF_same_last_name(c):
    p1_last_name = last_name(c.person1.get_span())
    p2_last_name = last_name(c.person2.get_span())
    if p1_last_name and p2_last_name and p1_last_name == p2_last_name:
        if c.person1.get_span() != c.person2.get_span():
            return (1,1)
    return (0,0)

def LF_no_spouse_in_sentence(c):
    return (-1,1) if np.random.rand() < 0.75 and len(spouses.intersection(c.get_parent().words)) == 0 else (0,0)

def LF_and_married(c):
    return (1,1) if 'and' in get_between_tokens(c) and 'married' in get_right_tokens(c) else (0,0)
    
def LF_familial_relationship(c):
    return (-1,1) if len(family.intersection(get_between_tokens(c))) > 0 else (0,0)

def LF_family_left_window(c):
    if len(family.intersection(get_left_tokens(c[0], window=2))) > 0:
        return (-1,1)
    elif len(family.intersection(get_left_tokens(c[1], window=2))) > 0:
        return (-1,1)
    else:
        return (0,0)

def LF_other_relationship(c):
    return (-1,1) if len(other.intersection(get_between_tokens(c))) > 0 else (0,0)


import bz2

# Function to remove special characters from text
def strip_special(s):
    return ''.join(c for c in s if ord(c) < 128)

# Read in known spouse pairs and save as set of tuples
with bz2.BZ2File('data/spouses_dbpedia.csv.bz2', 'rb') as f:
    known_spouses = set(
        tuple(strip_special(x.decode('utf-8')).strip().split(',')) for x in f.readlines()
    )
# Last name pairs for known spouses
last_names = set([(last_name(x), last_name(y)) for x, y in known_spouses if last_name(x) and last_name(y)])
    
def LF_distant_supervision(c):
    p1, p2 = c.person1.get_span(), c.person2.get_span()
    return (1,1) if (p1, p2) in known_spouses or (p2, p1) in known_spouses else (0,0)

def LF_distant_supervision_last_names(c):
    p1, p2 = c.person1.get_span(), c.person2.get_span()
    p1n, p2n = last_name(p1), last_name(p2)
    return (1,1) if (p1 != p2) and ((p1n, p2n) in last_names or (p2n, p1n) in last_names) else (0,0)


LFs = [
    LF_distant_supervision, LF_distant_supervision_last_names, 
    LF_husband_wife, LF_husband_wife_left_window, LF_same_last_name,
    LF_no_spouse_in_sentence, LF_and_married, LF_familial_relationship, 
    LF_family_left_window, LF_other_relationship
]

In [15]:
''' output:

    [[[L_x1],[S_x1]],
     [[L_x2],[S_x2]],
     ......
     ......
    ]

'''
def get_L_S_Tensor(cands): 
    
    L_S = []
    for i,ci in enumerate(cands):
        L_S_ci=[]
        L=[]
        S=[]
        P_ik = []
        for LF in LFs:
            #print LF.__name__
            l,s = LF(ci)
            L.append(l)
            S.append((s+1)/2)  #to scale scores in [0,1] 
        L_S_ci.append(L)
        L_S_ci.append(S)
        L_S.append(L_S_ci) 
        if(i%5000 == 0 and i!=0):
            print(str(i)+'data points labelled in',(time.time() - start_time)/60,'mins')
    return L_S


In [16]:
# import matplotlib.pyplot as plt
import time
import numpy as np
start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

dev_L_S = get_L_S_Tensor(dev_cands)
np.save("dev_L_S_discrete",np.array(dev_L_S))

train_L_S = get_L_S_Tensor(train_cands)
np.save("train_L_S_discrete",np.array(train_L_S))

print("--- %s seconds ---" % (time.time() - start_time))

# test_L_S = get_L_S_Tensor(test_cands)
# pkl.dump(test_L_S,open("test_L_S.p","wb"))

started at: 5-5-2018, 15:4:18
5000data points labelled in 0.18987296819686889 mins
10000data points labelled in 0.32245505253473916 mins
15000data points labelled in 0.6370007793108622 mins
20000data points labelled in 0.9320659041404724 mins
--- 63.827953577041626 seconds ---


In [17]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

def draw2DArray(a):
    fig = plt.figure(figsize=(6, 3.2))
    ax = fig.add_subplot(111)
    ax.set_title('colorMap')
    plt.imshow(np.array(a))
    ax.set_aspect('equal')
    cax = fig.add_axes([0.12, 0.1, 0.78, 0.8])
    cax.get_xaxis().set_visible(False)
    cax.get_yaxis().set_visible(False)
    cax.patch.set_alpha(0)
    cax.set_frame_on(False)
    plt.colorbar(orientation='vertical')
    plt.show()
    
      
def report2dict(cr):
    # Parse rows
    tmp = list()
    for row in cr.split("\n"):
        parsed_row = [x for x in row.split("  ") if len(x) > 0]
        if len(parsed_row) > 0:
            tmp.append(parsed_row)
    
    # Store in dictionary
    measures = tmp[0]

    D_class_data = defaultdict(dict)
    for row in tmp[1:]:
        class_label = row[0]
        for j, m in enumerate(measures):
            D_class_data[class_label][m.strip()] = float(row[j + 1].strip())
    return pd.DataFrame(D_class_data).T

def predictAndPrint(pl):
    print("acc",accuracy_score(gold_labels_dev,pl))
#     print(precision_recall_fscore_support(true_labels,pl,average='macro'))
    print(confusion_matrix(gold_labels_dev,pl))
    draw2DArray(confusion_matrix(gold_labels_dev,pl))
    return report2dict(classification_report(gold_labels_dev, pl))# target_names=class_names))
    
  

In [18]:
import numpy as np
dev_L_S = np.load("dev_L_S_discrete.npy")
train_L_S = np.load("train_L_S_discrete.npy")
print(dev_L_S.shape,train_L_S.shape)

(2814, 2, 10) (22276, 2, 10)


In [19]:
NoOfLFs= len(LFs)
NoOfClasses = 2

In [20]:
LF_l = [
    1,1,1,1,1,-1,1,-1,-1,-1
]

In [27]:
## class wise reproduced using map over y
from __future__ import absolute_import, division, print_function

import tensorflow as tf
BATCH_SIZE = 1
tf.reset_default_graph()

seed = 12
with tf.Graph().as_default():
    
    train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
    dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(dev_L_S.shape[0])

    labels = tf.convert_to_tensor(gold_labels_dev)
    iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
    next_element = iterator.get_next()

    train_init_op = iterator.make_initializer(train_dataset)
    dev_init_op = iterator.make_initializer(dev_dataset)
   
    next_element = iterator.get_next()
    print("next_element",next_element)

    alphas = tf.get_variable('alphas', [NoOfLFs],\
                             initializer=tf.truncated_normal_initializer(0.2,0.1,seed),\
                             dtype=tf.float64)

    thetas = tf.get_variable('thetas', [1,NoOfLFs],\
                             initializer=tf.truncated_normal_initializer(1,0.1,seed),\
                             dtype=tf.float64)
    
    k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
    print("k",k)
    print(alphas.graph)
    print(thetas.graph)
    l,s =  tf.unstack(next_element,axis=1)
    print(alphas)
    print(s)
    print("l",l)
    print(s.graph)

    s_ = tf.map_fn(lambda x : tf.maximum(tf.subtract(x,alphas), 0), s)

            
        
    ls_ = tf.multiply(l,s_)

    nls_ = tf.multiply(l,s_)*-1
    
    pout = tf.map_fn(lambda x: l*x,np.array([-1,1],dtype=np.float64))
    print("nls",nls_)
    print("thetas",thetas)
    
#     lst = tf.matmul(ls_,thetas)
#     print("lst",lst)
    t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout)
    print("pout",pout)
    
    print("t_pout",t_pout)

    t_k =  k*tf.squeeze(thetas)
    print("t_k",t_k)
    
    zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t_k*y),axis=0),np.array([-1,1],dtype=np.float64))
    
    logz = tf.log(tf.reduce_sum(zy,axis=0))
    print("zy",zy)
    print("logz",logz)
    
    lsp = tf.reduce_logsumexp(t_pout)
    print("lsp",lsp)

#     normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0)) - logz)  # add z

    normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0) - logz))


    print("normloss",normloss)
    marginals = tf.nn.softmax(t_pout,axis=0)
      
    print("marginals",marginals)
    predict = tf.argmax(marginals,axis=0)
    print("predict",predict)
    
#     pre = tf.metrics.precision(labels,predict)
#     rec = tf.metrics.recall(labels,predict)
#     print("loss",loss)
#     print("nls_",nls_)

#     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
#     starter_learning_rate = 1.0
#     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
#                                            10, 0.96, staircase=True)
#     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


#     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
#     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
#     reg_constant = 5.0  # Choose an appropriate one.
#     totalloss = normloss + reg_constant * sum(reg_losses)
    train_step = tf.train.AdamOptimizer(0.01).minimize(normloss) 
#     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
#     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

#     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

    init_g = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()
    with tf.Session() as sess:
        sess.run(init_g)
        sess.run(init_l)
        
        # Initialize an iterator over the training dataset.
        for it in range(5):
            sess.run(train_init_op)
            tl = 0
            try:
                while True:
                    _,ls = sess.run([train_step,normloss])
                    tl = tl + ls
            except tf.errors.OutOfRangeError:
                pass
            print(it,"loss",tl)
            
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)
          
                      
#         MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())
            

            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
            print()
            
        # Initialize an iterator over the validation dataset.
        sess.run(dev_init_op)
        a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
        print(a)
        print(t)
                      
        MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())
    
       
        unique, counts = np.unique(pl, return_counts=True)
        print(dict(zip(unique, counts)))
       
        print("acc",accuracy_score(gold_labels_dev,pl))
  
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl)))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="macro"))
        cf = confusion_matrix(gold_labels_dev,pl)
        print(cf)
        print("prec: tp/(tp+fp)",cf[1][1]/(cf[1][1]+cf[0][1]),"recall: tp/(tp+fn)",cf[1][1]/(cf[1][1]+cf[1][0]))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))

    
      
            


next_element Tensor("IteratorGetNext_1:0", shape=(?, 2, 10), dtype=float64)
k Tensor("Const_1:0", shape=(10,), dtype=float64)
<tf.Variable 'alphas:0' shape=(10,) dtype=float64_ref>
Tensor("unstack:1", shape=(?, 10), dtype=float64)
l Tensor("unstack:0", shape=(?, 10), dtype=float64)
nls Tensor("mul:0", shape=(?, 10), dtype=float64)
thetas <tf.Variable 'thetas:0' shape=(1, 10) dtype=float64_ref>
pout Tensor("map_1/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("map_2/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t_k Tensor("mul_1:0", shape=(10,), dtype=float64)
zy Tensor("map_3/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("Log:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(), dtype=float64)
normloss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
predict Tensor("ArgMax:0", shape=(?, 1), dtype=int64)
0 lo